In [ ]:
"""
Created on Mon Jan 20 11:50 2025

Compare the file currently used for IPSL-CM7 and the one I created to make them look similar

@author: Clara Burgard
"""

In [64]:
import xarray as xr
from multimelt.constants import *
import numpy as np
import matplotlib.pyplot as plt

In [41]:
%matplotlib qt5

QStandardPaths: error creating runtime directory '/run/user/2784' (Permission denied)


READ DATA

In [16]:
inputpath_raw='/data/cburgard/PREPARE_FORCING/PREPARE_PRESCRIBED_MELT/raw/'
inputpath_interim='/data/cburgard/PREPARE_FORCING/PREPARE_PRESCRIBED_MELT/interim/'
inputpath_raw2 = '/data/cburgard/PREPARE_FORCING/PREPARE_CAVITY_MASKS/raw/'


In [4]:
file_IPSL = xr.open_dataset(inputpath_raw + 'eORCA1.4.2_runoff-icb_DaiTrenberth_Depoorter.nc')
file_Davison = xr.open_dataset(inputpath_interim + 'Davison23_basal_melt_pattern_on_eORCA1.nc')
file_current = xr.open_dataset(inputpath_interim + 'masks_for_eORCA1_prescribedmeltinopencav.nc')

In [17]:
domain_cfg = xr.open_dataset(inputpath_raw2 + 'eORCA1.4.3_OpenSeas_OpenAllCav_ModStraights/eORCA1.4.3_OpenSeas_OpenAllCav_ModStraights_domain_cfg.nc')

In [18]:
cell_area = domain_cfg['e1t'] * domain_cfg['e2t']

In [37]:
melt_converted = file_current['melt_isf'] / yearinsec *10**(12) / cell_area 

In [50]:
melt_IPSL_Gt = file_IPSL['sornfisf'].isel(time_counter=0) * yearinsec * 10**(-12) * cell_area

In [51]:
melt_IPSL_Gt.plot()

In [62]:
melt_IPSL_Gt.where(melt_IPSL_Gt != 0).plot(vmin=-5,vmax=5,cmap = 'coolwarm')

In [65]:
plt.figure()
(file_current['melt_isf']).plot(vmin=-5,vmax=5,cmap = 'coolwarm')

In [46]:
(melt_converted - file_IPSL['sornfisf'].isel(time_counter=0))/file_IPSL['sornfisf'].isel(time_counter=0)).plot(vmax=1)

NameError: name 'np' is not defined

In [66]:
(file_current['melt_isf'] - melt_IPSL_Gt).plot()

In [68]:
melt_converted.plot()

In [85]:
modified_file_IPSL = file_IPSL.copy()

In [113]:
modified_file_IPSL['sornfisf'] = file_IPSL['sornfisf'] * 0 + melt_converted.squeeze()
modified_file_IPSL['sornfisf'].attrs['long_name'] = 'runoff from antarctica iceshelves (Davison 2023)'

In [115]:
modified_file_IPSL['sodepmax_isf'] = file_IPSL['sodepmax_isf'] * 0 + file_current['z_max'].squeeze()
modified_file_IPSL['sodepmin_isf'] = file_IPSL['sodepmin_isf'] * 0 + file_current['z_min'].squeeze()

In [116]:
modified_file_IPSL.attrs['description2'] = 'south of 60 south : Davison Freshwater Discharge from basal front to grounding line iceshelves melting'
modified_file_IPSL.attrs['reference3'] = 'B.J. Davison, A.E. Hogg, N. Gourmelen, L. Jakob, J. Wuite, T. Nagler, C.A. Greene, J. Andreasen, M.E. Engdahl, 2023: Annual mass budget of Antarctic ice shelves from 1997 to 2021. Science Advances, doi:10.1126/sciadv.adi0186'
modified_file_IPSL.attrs['creation'] = 'Mon Jan 20 16:11 2025'

In [117]:
modified_file_IPSL.attrs['history5'] = 'modified from eORCA1.4.2_runoff-icb_DaiTrenberth_Depoorter.nc by replacing the ice shelf freshwater flux by values from Davison and prescribing the flux in large cavities to open them in NEMO (contact Clara Burgard)' 

In [118]:
file_IPSL

<xarray.Dataset>
Dimensions:       (y: 331, x: 360, time_counter: 12)
Coordinates:
    nav_lon       (y, x) float32 ...
    nav_lat       (y, x) float32 ...
  * time_counter  (time_counter) int32 1 2 3 4 5 6 7 8 9 10 11 12
Dimensions without coordinates: y, x
Data variables:
    sorunoff      (time_counter, y, x) float32 ...
    sornfisf      (time_counter, y, x) float32 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    socoefr       (y, x) float32 ...
    sodepmax_isf  (time_counter, y, x) float32 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    sodepmin_isf  (time_counter, y, x) float32 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    Icb_flux      (time_counter, y, x) float32 ...
Attributes: (12/14)
    title:         global runoff projected on eORCA1 grid
    description1:  north of 60 south : Dai_Trenberth Continental Freshwater D...
    description2:  south of 60 south : Depoorter Freshwater Discharge from ba...
    description3:  Merino I. Freshwater flux from Iceberg melting in Southern...
    history1:      north of 60 south : use runoff_1m_DaiTrenberthSummer_1951_...
    history2:      south of 60 south : use Depoorter estimates (contact Ignac...
    ...            ...
    reference3:    M. A. Depoorter, J. L. Bamber, J. A. Griggs, J. T. M. Lena...
    creation:      Mon Dec 14 16:31:35 2015
    history3:      socoefr north of 60 south from eORCA1_R1_runoff_clim_v1.0_...
    history4:      sorunoff from eORCA1_R1_runoff_clim_v1.0_nomask.nc (north ...
    history:       Mon Nov 23 19:15:48 2020: ncks -O -F --abc -d x,2,361 -d y...
    NCO:           netCDF Operators version 4.8.1 (Homepage = http://nco.sf.n...

In [120]:
modified_file_IPSL.to_netcdf(inputpath_interim + 'eORCA1.4.2_runoff-icb_DaiTrenberth_Davison.nc')

In [102]:
(modified_file_IPSL['sodepmax_isf'] - file_IPSL['sodepmax_isf']).isel(time_counter=0).plot()

In [ ]:
modified_file_IPSL['sodepmax_isf'].plot()